This example shows you the basic usage of the windpowerlib. 
There are mainly three steps. First you have to import your weather data, then you need to specify your wind turbine, and in the last step call the windpowerlib functions to calculate the feedin timeseries.

### Import necessary packages and modules

In [ ]:
__copyright__ = "Copyright oemof developer group"
__license__ = "GPLv3"

import os
import pandas as pd

from windpowerlib import modelchain
from windpowerlib import wind_turbine as wt

You can use the logging package to get logging messages from the windpowerlib. Change the logging level if you want more or less messages.

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

### Import weather data

In order to use the windpowerlib you need to at least provide wind speed data for the time frame you want to analyse.
The function below imports example weather data from the weather.csv file provided along with the windpowerlib. The data include wind speed at two different heights in m/s, air temperature in two different heights in K, surface roughness length in m and air pressure in Pa.

To find out which weather data in which units need to be provided in order to use the ModelChain or other functions of the windpowerlib see the individual function documentation.

In [ ]:
def read_weather_data(filename, datetime_column='time_index',
                      **kwargs):
    r"""
    Fetches weather data from a file.

    The files are located in the example folder of the windpowerlib.

    Parameters
    ----------
    filename : string
        Filename of the weather data file.
    datetime_column : string
        Name of the datetime column of the weather DataFrame.

    Other Parameters
    ----------------
    datapath : string, optional
        Path where the weather data file is stored.
        Default: 'windpowerlib/example'.

    Returns
    -------
    pandas.DataFrame
        Contains weather data time series.

    """
    if 'datapath' not in kwargs:
        kwargs['datapath'] = os.path.join(os.path.split(
            os.path.dirname(__file__))[0], 'example')

    file = os.path.join(kwargs['datapath'], filename)
    df = pd.read_csv(file)
    return df.set_index(pd.to_datetime(df[datetime_column])).tz_localize(
        'UTC').tz_convert('Europe/Berlin').drop(datetime_column, 1)


# Read weather data from csv
weather = read_weather_data(filename='weather.csv', datapath='')

Along with the weather data you have to provide a dataframe or dictionary specifying the height for which the data applies.

In [ ]:
# height in meters
data_height = {
    'pressure': 0,
    'temp_air': 2,
    'v_wind': 10,
    'temp_air_2': 10,
    'v_wind_2': 80}

### Initialise wind turbine

To initialise a specific turbine you need a dictionary that contains the basic parameters. A turbine is defined by its nominal power, hub height, rotor diameter, and power or power coefficient curve.

There are two ways to initialise a WindTurbine object in the windpowerlib. You can either specify your own turbine, as done below for 'myTurbine', or fetch power and/or power coefficient curve data from data files provided by the windpowerlib, as done for the 'enerconE126'.

You can execute the following to get a list of all wind turbines for which power or power coefficient curves are provided.

In [ ]:
# get power curves
# print names of wind turbines for which power curves are provided (default)
wt.get_turbine_types()

# get power coefficient curves
# write names of wind turbines for which power coefficient curves are provided
# to 'turbines' DataFrame
turbines = wt.get_turbine_types(filename='cp_curves.csv', print_out=False)
# find all Enercons in 'turbines' DataFrame
print(turbines[turbines["turbine_id"].str.contains("ENERCON")])

In [ ]:
# specification of own wind turbine (Note: power coefficient values and
# nominal power have to be in Watt)
myTurbine = {
    'turbine_name': 'myTurbine',
    'nominal_power': 3e6,  # in W
    'hub_height': 105,  # in m
    'd_rotor': 90,  # in m
    'p_values': pd.DataFrame(
        data={'p': [p * 1000 for p in
                    [0.0, 26.0, 180.0, 1500.0, 3000.0, 3000.0]]},  # in W
        index=[0.0, 3.0, 5.0, 10.0, 15.0, 25.0])  # in m/s
    }  
# initialise WindTurbine object
my_turbine = wt.WindTurbine(**myTurbine)

In [ ]:
# specification of wind turbine where power curve is provided
# if you want to use the power coefficient curve add {'fetch_curve': 'cp'}
# to the dictionary
enerconE126 = {
    'turbine_name': 'ENERCON E 126 7500',  # turbine name as in register
    'hub_height': 135,  # in m
    'd_rotor': 127  # in m
    }
# initialise WindTurbine object
e126 = wt.WindTurbine(**enerconE126)

### Use the ModelChain to generate feedin timeseries

The ModelChain is a class that provides all necessary steps to calculate the power output of a wind turbine. If you use the 'run_model' method first the wind speed and density (if necessary) at hub height are calculated and then used to calculate the power output. You can either use the default methods for the calculation steps, as done for 'my_turbine', or choose different methods, as done for the 'e126'.

In [ ]:
# power output calculation for my_turbine
# initialise ModelChain with default parameters and use run_model
# method to calculate power output
mc_my_turbine = modelchain.ModelChain(my_turbine).run_model(
    weather, data_height)
# write power output timeseries to WindTurbine object
my_turbine.power_output = mc_my_turbine.power_output

In [ ]:
# power output calculation for e126
# own specifications for ModelChain setup
modelchain_data = {
    'obstacle_height': 0,              # default: 0
    'wind_model': 'logarithmic',       # 'logarithmic' (default) or 'hellman'
    'rho_model': 'ideal_gas',          # 'barometric' (default) or 'ideal_gas'
    'power_output_model': 'p_values',  # 'p_values' (default) or 'cp_values'
    'density_corr': True,              # False (default) or True
    'hellman_exp': None}               # None (default) or None
# initialise ModelChain with own specifications and use run_model method to
# calculate power output
mc_e126 = modelchain.ModelChain(e126, **modelchain_data).run_model(
    weather, data_height)
# write power output timeseries to WindTurbine object
e126.power_output = mc_e126.power_output

### Plot results

If you have matplotlib installed you can visualise the calculated power output and used power (coefficient) curves.

In [ ]:
# try to import matplotlib
try:
    from matplotlib import pyplot as plt
    # matplotlib inline needed in notebook to plot inline
    %matplotlib inline 
except ImportError:
    plt = None

In [ ]:
# plot turbine power output
if plt:
    e126.power_output.plot(legend=True, label='Enercon E126')
    my_turbine.power_output.plot(legend=True, label='myTurbine')
    plt.show()

In [ ]:
# plot power (coefficient) curves
if plt:
    if e126.cp_values is not None:
        e126.cp_values.plot(style='*', title='Enercon E126')
        plt.show()
    if e126.p_values is not None:
        e126.p_values.plot(style='*', title='Enercon E126')
        plt.show()
    if my_turbine.cp_values is not None:
        my_turbine.cp_values.plot(style='*', title='myTurbine')
        plt.show()
    if my_turbine.p_values is not None:
        my_turbine.p_values.plot(style='*', title='myTurbine')
        plt.show()